In [ ]:
#ref : https://pypi.org/project/onnx2tf/
!sudo apt-get -y update
!sudo apt-get -y install python3-pip
!sudo apt-get -y install python-is-python3
!wget https://github.com/PINTO0309/onnx2tf/releases/download/1.16.31/flatc.tar.gz \
  && tar -zxvf flatc.tar.gz \
  && sudo chmod +x flatc \
  && sudo mv flatc /usr/bin/
!pip install -U pip \
  && pip install tensorflow==2.15.0 \
  && pip install -U onnx==1.15.0 \
  && python -m pip install onnx_graphsurgeon \
        --index-url https://pypi.ngc.nvidia.com \
  && pip install -U onnxruntime==1.16.3 \
  && pip install -U onnxsim==0.4.33 \
  && pip install -U simple_onnx_processing_tools \
  && pip install -U onnx2tf \
  && pip install -U protobuf==3.20.3 \
  && pip install -U h5py==3.7.0 \
  && pip install -U psutil==5.9.5 \
  && pip install -U ml_dtypes==0.2.0

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [662 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,600 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,621 kB]
Get:13 http://s

In [ ]:
import math
from google.colab import drive
drive.mount("/content/drive")
from PIL import Image
import requests
import matplotlib.pyplot as plt
import tensorflow
print (tensorflow.__version__)
import ipywidgets as widgets
from IPython.display import display, clear_output

import torch
from torch import nn
from torchvision.models import resnet50
import torchvision.transforms as T
torch.set_grad_enabled(False);
model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
model.eval();

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2.15.0


Using cache found in /root/.cache/torch/hub/facebookresearch_detr_main
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
torch.onnx.export(model, torch.rand(1,3,224,224), "/content/drive/MyDrive/detr.onnx", opset_version=12)

/root/.cache/torch/hub/facebookresearch_detr_main/util/misc.py:338: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  max_size_i = torch.max(torch.stack([img.shape[i] for img in tensor_list]).to(torch.float32)).to(torch.int64)
/root/.cache/torch/hub/facebookresearch_detr_main/util/misc.py:348: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for img in tensor_list:


In [ ]:
!onnx2tf -i /content/drive/MyDrive/detr.onnx -osd

Streaming output truncated to the last 5000 lines.
INFO:  input.1.a: name: tf.math.add_210/Add:0 shape: (49, 1, 256) dtype: <dtype: 'float32'> 
INFO:  input.2.b: shape: (256, 256) dtype: float32 
INFO:  input.3.output_type: name: float32 shape: () 
INFO:  output.1.output: name: tf.linalg.matmul_101/MatMul:0 shape: (49, 1, 256) dtype: <dtype: 'float32'> 

INFO: 515 / 1008
INFO: onnx_op_type: MatMul onnx_op_name: wa/transformer/decoder/layers.0/multihead_attn/MatMul_1
INFO:  input_name.1: wa/transformer/decoder/layers.0/Add_3_output_0 shape: [49, 1, 256] dtype: float32
INFO:  input_name.2: onnx::MatMul_5373 shape: [256, 256] dtype: float32
INFO:  output_name.1: wa/transformer/decoder/layers.0/multihead_attn/MatMul_1_output_0 shape: [49, 1, 256] dtype: float32
INFO: tf_op_type: matmul
INFO:  input.1.a: name: tf.math.add_211/Add:0 shape: (49, 1, 256) dtype: <dtype: 'float32'> 
INFO:  input.2.b: shape: (256, 256) dtype: float32 
INFO:  input.3.output_type: name: float32 shape: () 
INFO:  ou

In [ ]:
!cp -r saved_model /content/drive/MyDrive/saved_model/

In [ ]:
!ls  saved_model /content/drive/MyDrive/saved_model/

/content/drive/MyDrive/saved_model/:
assets	detr_float16.tflite  detr_float32.tflite  fingerprint.pb  saved_model.pb  variables

saved_model:
assets	detr_float16.tflite  detr_float32.tflite  fingerprint.pb  saved_model.pb  variables


In [ ]:
!echo "deb [signed-by=/usr/share/keyrings/coral-edgetpu-archive-keyring.gpg] https://packages.cloud.google.com/apt coral-edgetpu-stable main"
! sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg
!sudo tee /usr/share/keyrings/coral-edgetpu-archive-keyring.gpg >/dev/null
!sudo apt-get update

!sudo apt-get install python3-tflite-runtime libatlas-base-dev



deb [signed-by=/usr/share/keyrings/coral-edgetpu-archive-keyring.gpg] https://packages.cloud.google.com/apt coral-edgetpu-stable main
-----BEGIN PGP PUBLIC KEY BLOCK-----

mQENBGKItdQBCADWmKTNZEYWgXy73FvKFY5fRro4tGNa4Be4TZW3wZpct9Cj8Ejy
kU7S9EPoJ3EdKpxFltHRu7QbDi6LWSNA4XxwnudQrYGxnxx6Ru1KBHFxHhLfWsvF
cGMwit/znpxtIt9UzqCm2YTEW5NUnzQ4rXYqVQK2FLG4weYJ5bKwkY+ZsnRJpzxd
HGJ0pBiqwkMT8bfQdJymUBown+SeuQ2HEqfjVMsIRe0dweD2PHWeWo9fTXsz1Q5a
biGckyOVyoN9//DgSvLUocUcZsrWvYPaN+o8lXTO3GYFGNVsx069rxarkeCjOpiQ
OWrQmywXISQudcusSgmmgfsRZYW7FDBy5MQrABEBAAG0UVJhcHR1cmUgQXV0b21h
dGljIFNpZ25pbmcgS2V5IChjbG91ZC1yYXB0dXJlLXNpZ25pbmcta2V5LTIwMjIt
MDMtMDctMDhfMDFfMDEucHViKYkBIgQTAQgAFgUCYoi11AkQtT3IDRPt7wUCGwMC
GQEAAMGoB/98QBNIIN3Q2D3aahrfkb6axd55zOwR0tnriuJRoPHoNuorOpCv9aWM
MvQACNWkxsvJxEF8OUbzhSYjAR534RDigjTetjK2i2wKLz/kJjZbuF4ZXMynCm40
eVm1XZqU63U9XR2RxmXppyNpMqQO9LrzGEnNJuh23icaZY6no12axymxcle/+SCm
da8oDAfa0iyA2iyg/eU05buZv54MC6RB13QtS+8vOrKDGr7RYp/VYvQzYWm+ck6D
vlaVX6VB51BkLl23SQknyZIJBVPm8ttU65EyrrgG1jLLHFXD

In [ ]:
import tensorflow as tf
from google.colab import drive
drive.mount("/content/drive/")
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/saved_model/detr_float16.tflite")
interpreter.allocate_tensors()

Mounted at /content/drive/


In [ ]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Check input and output shapes
print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])

Input shape: [  1 224 224   3]
Output shape: [  1 100  92]
